# Kaggle 持久化开发环境工作流

本 Notebook 结合了 **Kaggle Notebook** 和 **GitHub**，旨在创建一个稳定、可复现且不会因会话中断而丢失工作的开发环境。
Kaggle本身支持持久化，开启设置即可，所以我们可以将所有文件保存。
但是Kaggle不支持apt安装命令，不能安装C++程序，依赖的TensorRT等库也无法安装，只能通过数据集上传/挂载。

**核心思想:**
1.  **Kaggle Notebook**: 作为主要的开发环境，运行和查看结果。所有文件都保存在Kaggle的持久化存储中。
2.  **GitHub**: 作为我们代码的“版本控制系统”，用于追踪和管理所有代码 (`.py`, `.ipynb`) 和配置文件。


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#!rm -rf trt-yolov8-accelerator

In [3]:
!git clone https://github.com/libingwei/AI-Infer-Acc.git

Cloning into 'AI-Infer-Acc'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 194 (delta 93), reused 170 (delta 69), pack-reused 0 (from 0)
Receiving objects: 100% (194/194), 78.65 KiB | 1.83 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [2]:
cd /kaggle/working/AI-Infer-Acc

/kaggle/working/AI-Infer-Acc


In [ ]:
!git fetch
!git pull
# !git submodule sync

In [ ]:
!git log

In [13]:
!git submodule update --init --recursive
!git submodule update

Submodule path 'projects/trt-yolov8-accelerator': checked out 'a8350b3106fde92c46315314e8cfa7ce24f16cd5'


In [5]:
!pwd

/kaggle/working/AI-Infer-Acc


In [8]:
!nvidia-smi
!nvcc --version

Tue Aug 12 04:18:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [18]:
%%bash
# ls -la /kaggle/input/tensorrt-10-4-0-26-cuda-12-6/TensorRT-10.4.0.26/include
# ls -l /kaggle/input/tensorrt-10-4-0-26-cuda-12-6/TensorRT-10.4.0.26/targets/x86_64-linux-gnu/lib/
# ls -l /usr/lib/x86_64-linux-gnu/libcud*

# cp -r /kaggle/input/tensorrt-10-4-0-26-cuda-12-6/TensorRT-10.4.0.26/targets/x86_64-linux-gnu/lib build/lib
# ls -l build/lib
# bash scripts/fix_trt_symlinks.sh build/lib
# ls -l build/lib
# mkdir -p /kaggle/working/tensorrt/
# cp -r /kaggle/input/tensorrt-10-4-0-26-cuda-12-6/TensorRT-10.4.0.26/include  /kaggle/working/tensorrt/include
# mv build/lib /kaggle/working/tensorrt/lib

bash scripts/fix_trt_symlinks.sh /kaggle/working/tensorrt/lib
ls -l /kaggle/working/tensorrt/include
ls -l /kaggle/working/tensorrt/lib

Fixing symlinks in: /kaggle/working/tensorrt/lib
  libnvonnxparser.so             -> libnvonnxparser.so.10.4.0
  libnvonnxparser.so.10          -> libnvonnxparser.so.10.4.0
  libnvinfer_lean.so             -> libnvinfer_lean.so.10.4.0
  libnvinfer_lean.so.10          -> libnvinfer_lean.so.10.4.0
  libnvinfer.so                  -> libnvinfer.so.10.4.0
  libnvinfer.so.10               -> libnvinfer.so.10.4.0
  libnvinfer_plugin.so           -> libnvinfer_plugin.so.10.4.0
  libnvinfer_plugin.so.10        -> libnvinfer_plugin.so.10.4.0
  libnvinfer_builder_resource.so -> libnvinfer_builder_resource.so.10.4.0
  libnvinfer_builder_resource.so.10 -> libnvinfer_builder_resource.so.10.4.0
  libnvinfer_vc_plugin.so        -> libnvinfer_vc_plugin.so.10.4.0
  libnvinfer_vc_plugin.so.10     -> libnvinfer_vc_plugin.so.10.4.0
  libnvinfer_builder_resource_win.so -> libnvinfer_builder_resource_win.so.10.4.0
  libnvinfer_builder_resource_win.so.10 -> libnvinfer_builder_resource_win.so.10.4.0

Done. To

In [4]:
%%bash
git fetch && git pull

Already up to date.


In [ ]:
%%bash
rm -rf build
cmake -S . -B build -DCMAKE_BUILD_TYPE=Debug \
  -DTENSORRT_INCLUDE_DIRS=/kaggle/working/tensorrt/include \
  -DTENSORRT_LIBRARY_DIRS=/kaggle/working/tensorrt/lib
cmake --build build -j --verbose -t simple_trt_test |& tail -n 200

In [ ]:
!cmake --build build -j

In [ ]:
# %%bash
# echo "libnvinfer_builder_resource.so.10.4.0"
# ldd /kaggle/working/AI-Infer-Acc/build/bin/libnvinfer_builder_resource.so.10.4.0
# echo "libnvinfer_plugin.so.10.4.0"
# ldd /kaggle/working/AI-Infer-Acc/build/bin/libnvinfer_plugin.so.10.4.0

In [ ]:
%%bash
# cd /kaggle/working/AI-Infer-Acc
# #python scripts/generate_onnx_model.py
# python scripts/download_calibration_data.py

# ls -la /kaggle/working/AI-Infer-Acc/models
# ls -la /kaggle/working/AI-Infer-Acc/calibration_data


# cd /kaggle/working/AI-Infer-Acc

In [ ]:
%%bash
# ls -la build/bin
# cd /kaggle/working/AI-Infer-Acc/build/bin
cd /kaggle/working/AI-Infer-Acc
export LD_LIBRARY_PATH="/kaggle/working/tensorrt/lib:${LD_LIBRARY_PATH}"

# ldd bin/simple_trt_test | cat   # 再看一眼依赖
# build/bin/simple_trt_test
# ldd libtrt_utils.so
build/bin/onnx_to_trt models/resnet18.onnx models/resnet18 int8 /kaggle/working/AI-Infer-Acc/calibration_data

cd /kaggle/working/AI-Infer-Acc

In [60]:
%%bash
cd /kaggle/working/AI-Infer-Acc
export LD_LIBRARY_PATH="/kaggle/working/tensorrt/lib:${LD_LIBRARY_PATH}"

build/bin/trt_inference models/resnet18.trt 32 
build/bin/trt_inference models/resnet18_fp16.trt 32 
build/bin/trt_inference models/resnet18_int8.trt 32 
build/bin/trt_inference models/resnet18_int8.trt 32 --use-default-stream

cd /kaggle/working/AI-Infer-Acc

Running inference in FP32 mode.
Using Batch Size: 32

--- Running Performance Test ---
Number of iterations: 100

--- Performance Results ---
Total time for 100 inferences: 2536.33 ms
Average Latency: 25.3633 ms
Throughput: 1261.66 FPS (frames per second)
---------------------------

Displaying first 10 output values from the last run:
-0.0391306 0.114466 -1.79676 -1.2343 -0.819005 0.323965 -2.1866 -1.28767 -1.90192 -0.73148 
Running inference in FP16 mode.
Using Batch Size: 32

--- Running Performance Test ---
Number of iterations: 100

--- Performance Results ---
Total time for 100 inferences: 861.688 ms
Average Latency: 8.61688 ms
Throughput: 3713.64 FPS (frames per second)
---------------------------

Displaying first 10 output values from the last run:
-0.0232391 0.118225 -1.79688 -1.2334 -0.8125 0.344727 -2.18164 -1.28418 -1.90039 -0.731445 
Running inference in INT8 mode.
Using Batch Size: 32

--- Running Performance Test ---
Number of iterations: 100

--- Performance Results --

In [58]:
%%bash
cd /kaggle/working/AI-Infer-Acc
export LD_LIBRARY_PATH="/kaggle/working/tensorrt/lib:${LD_LIBRARY_PATH}"

echo "trt_compare resnet18 vs resnet18_fp16"
build/bin/trt_compare models/resnet18.trt models/resnet18_fp16.trt calibration_data 200

echo "trt_compare resnet18 vs resnet18_int8"
build/bin/trt_compare models/resnet18.trt models/resnet18_int8.trt calibration_data 200

trt_compare resnet18 vs resnet18_fp16
Evaluating on 200 images (H=224, W=224)

=== Accuracy Consistency Report ===
Samples: 200
Top-1 Agreement: 98%
Top-5 Agreement: 100%
Avg Cosine Similarity: 0.999991
Avg L2 Distance: 0.327497
trt_compare resnet18 vs resnet18_int8
Evaluating on 200 images (H=224, W=224)

=== Accuracy Consistency Report ===
Samples: 200
Top-1 Agreement: 89%
Top-5 Agreement: 100%
Avg Cosine Similarity: 0.997317
Avg L2 Distance: 5.4276


In [10]:
%%bash
cd /kaggle/working/AI-Infer-Acc
# rm -rf datasets
# mkdir datasets
# cd datasets
# gdown 1hyU4n_KgOdUsgrG-4RcHqHs1iYRk_AKj # Your ID of ILSVRC2012_devkit_t12.tar.gz goes here
# gdown 14bUC__FMj77FQhMNyuOsWbW3qFaa3OkO # Your ID of ILSVRC2012_img_val.tar goes here
# cd ..
# mv ILSVRC2012_devkit_t12.tar.gz datasets/
# mv ILSVRC2012_img_val.tar datasets/
ls -l .
ls -l datasets
echo "imagenet_val"
ls -l imagenet_val/
# rm -rf imagenet_val
# mkdir -p imagenet_val/val
python scripts/prepare_imagenet_val.py --limit 1000

total 196
-rw-r--r-- 1 root root  8005 Aug 12 14:53 AI_Inference_Acceleration_Plan.md
drwxr-xr-x 6 root root  4096 Aug 12 14:53 apps
drwxr-xr-x 2 root root  4096 Aug 12 15:09 assets
drwxr-xr-x 7 root root  4096 Aug 12 15:09 build
drwxr-xr-x 3 root root 20480 Aug 12 14:53 calibration_data
-rw-r--r-- 1 root root  5822 Aug 12 14:53 CHANGELOG.md
drwxr-xr-x 2 root root  4096 Aug 12 14:54 cmake
-rw-r--r-- 1 root root 20159 Aug 12 14:53 CMakeLists.txt
drwxr-xr-x 2 root root  4096 Aug 12 14:54 datasets
drwxr-xr-x 3 root root  4096 Aug 12 14:54 examples
-rw-r--r-- 1 root root 24363 Aug 12 15:10 imagenet_classes.tsv
drwxr-xr-x 3 root root  4096 Aug 12 15:10 imagenet_val
-rw-r--r-- 1 root root 33893 Aug 12 15:10 imagenet_val_labels.csv
-rw-r--r-- 1 root root  2367 Aug 12 14:54 int8_calib_table.cache
drwxr-xr-x 3 root root  4096 Aug 12 14:54 libs
drwxr-xr-x 2 root root  4096 Aug 12 14:52 models
drwxr-xr-x 3 root root  4096 Aug 12 14:54 projects
-rw-r--r-- 1 root root 12835 Aug 12 15:09 README.md
-

In [14]:
! ls -l /kaggle/working/AI-Infer-Acc/imagenet_val/val |& head -n 5

total 133320
-rw-r--r-- 1 root root  109527 Jun 13  2012 ILSVRC2012_val_00000001.JPEG
-rw-r--r-- 1 root root  140296 Jun 13  2012 ILSVRC2012_val_00000002.JPEG
-rw-r--r-- 1 root root  122660 Jun 13  2012 ILSVRC2012_val_00000003.JPEG
-rw-r--r-- 1 root root   84885 Jun 13  2012 ILSVRC2012_val_00000004.JPEG


In [22]:
%%bash
cd /kaggle/working/AI-Infer-Acc
export LD_LIBRARY_PATH="/kaggle/working/tensorrt/lib:${LD_LIBRARY_PATH}"

ls /kaggle/working/AI-Infer-Acc/imagenet_val/val |& head -n 5
head -n 5 imagenet_val_labels.csv
echo "-----FP32 vs FP16-----"
build/bin/trt_compare models/resnet18.trt models/resnet18_fp16.trt imagenet_val/val 1000 --labels imagenet_val_labels.csv --class-names imagenet_classes.tsv --inspect 10 --center-crop --imagenet-norm

echo "-----FP32 vs INT8-----"
build/bin/trt_compare models/resnet18.trt models/resnet18_int8.trt imagenet_val/val 1000 --labels imagenet_val_labels.csv --class-names imagenet_classes.tsv --inspect 10 --center-crop --imagenet-norm

# ls /kaggle/working/AI-Infer-Acc/imagenet_val/

ILSVRC2012_val_00000001.JPEG
ILSVRC2012_val_00000002.JPEG
ILSVRC2012_val_00000003.JPEG
ILSVRC2012_val_00000004.JPEG
ILSVRC2012_val_00000005.JPEG
ILSVRC2012_val_00000001.JPEG,65
ILSVRC2012_val_00000002.JPEG,970
ILSVRC2012_val_00000003.JPEG,230
ILSVRC2012_val_00000004.JPEG,809
ILSVRC2012_val_00000005.JPEG,516
-----FP32 vs FP16-----
Evaluating on 1000 images (H=224, W=224)
Loaded labels from imagenet_val_labels.csv, entries: 1000
Loaded class names: 1000
[Inspect] ILSVRC2012_val_00000001.JPEG | FP32: 65:sea_snake | Test: 65:sea_snake | Top5(FP32): 65:sea_snake, 58:water_snake, 63:Indian_cobra, 62:rock_python, 54:hognose_snake | Top5(Test): 65:sea_snake, 58:water_snake, 63:Indian_cobra, 62:rock_python, 54:hognose_snake
[Inspect] ILSVRC2012_val_00000002.JPEG | FP32: 795:ski | Test: 795:ski | Top5(FP32): 795:ski, 970:alp, 792:shovel, 537:dogsled, 672:mountain_tent | Top5(Test): 795:ski, 970:alp, 792:shovel, 537:dogsled, 802:snowmobile
[Inspect] ILSVRC2012_val_00000003.JPEG | FP32: 230:Shetla

In [1]:
! head -n 10  imagenet_classes.tsv

head: cannot open 'imagenet_classes.tsv' for reading: No such file or directory


In [38]:
# !pip uninstall nvidia-pyindex
# !pip uninstall -y nvidia-tensorrt

Found existing installation: nvidia-tensorrt 99.0.0
Uninstalling nvidia-tensorrt-99.0.0:
  Successfully uninstalled nvidia-tensorrt-99.0.0
